In [1]:
import pandas as pd
import datasets

In [8]:
dartdataset = datasets.load_dataset('nlpHakdang/beneficiary',  data_files = "dart_ver1_2.csv", use_auth_token='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')

Using custom data configuration nlpHakdang___beneficiary-53bfe0868077b71a
Reusing dataset csv (/opt/ml/.cache/huggingface/datasets/csv/nlpHakdang___beneficiary-53bfe0868077b71a/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
newsdataset = datasets.load_dataset('nlpHakdang/aihub-news30k',  data_files = "news_ver_1_0.csv", use_auth_token='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')

Using custom data configuration nlpHakdang___aihub-news30k-a8e7ac7dd1cfd700
Reusing dataset csv (/opt/ml/.cache/huggingface/datasets/csv/nlpHakdang___aihub-news30k-a8e7ac7dd1cfd700/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/1 [00:00<?, ?it/s]

In [2]:
# news_data = pd.read_csv("../data/news/train/MainData.csv")
# #data2 = pd.read_csv("../data/news/train/SubData.csv")

In [3]:
# # Dart load 하기
# # with open("corp_2500_result.pickle", "rb") as fp:   # Unpickling
# #   dart_data = pickle.load(fp)
# dart = pd.read_csv('../data/dart/dart_v1_1_remove_context.csv')

In [4]:
dart_dict= {
    "corp_name" :[],
    "corp_text":[]
}

In [5]:
dic = eval(dart.iloc[0]['clean'])

In [6]:
dict_key={}
for key in dic.keys():
    dict_key[key] = dic[key].keys()

In [7]:
dict_key

{'I. 회사의 개요': dict_keys(['1. 회사의 개요', '2. 회사의 연혁']),
 'II. 사업의 내용': dict_keys(['1. 사업의 개요', '2. 주요 제품 및 서비스', '6. 주요계약 및 연구개발활동'])}

In [140]:
for idx in range(len(dart)):
    dart_text = ""
    dart_dict['corp_name'] = (dart['corp_name'])
    clean = eval(dart.iloc[idx]['clean'])
    for key in dict_key.keys():
        for k in dict_key[key]:
            if k == '2. 회사의 연혁':
                continue
            dart_text += clean[key][k]
    dart_dict['corp_text'].append(dart_text)
    

In [9]:
from konlpy.tag import Mecab

In [11]:
from konlpy.tag import Mecab
import operator
tokenizer = Mecab()

Exception: Install MeCab in order to use it: http://konlpy.org/en/latest/install/

In [15]:
import re 
def preprocessing(s): 
    hangul = re.compile('[^ ㄱ-ㅣ가-힣+]')
    result = hangul.sub('', s)
    return result


In [8]:
# dart_dict = {}
# error =0
# for dart in dart_data:
#     try:
#         dart_dict[test(dart['contents']['II. 사업의 내용']['1. 사업의 개요']) + " " +\
#             test(dart['contents']['II. 사업의 내용']['2. 주요 제품 및 서비스'])] = dart['기업 이름']
#     except:
#         error+=1

In [22]:
# dart_data[0]['contents'].keys()

dict_keys(['【 대표이사 등의 확인 】', 'I. 회사의 개요', 'II. 사업의 내용', 'III. 재무에 관한 사항', 'IV. 이사의 경영진단 및 분석의견', 'V. 회계감사인의 감사의견 등', 'VI. 이사회 등 회사의 기관에 관한 사항', 'VII. 주주에 관한 사항', 'VIII. 임원 및 직원 등에 관한 사항', 'IX. 계열회사 등에 관한 사항', 'X. 대주주 등과의 거래내용', 'XI. 그 밖에 투자자 보호를 위하여 필요한 사항', 'XII. 상세표', '【 전문가의 확인 】'])

In [144]:
dart_dict['corp_name'][2]

'케이비증권'

In [142]:
pre_dart = {}
for idx in range(len(dart_dict['corp_name'])):
    pre_dart[dart_dict['corp_name'][idx]] = preprocessing(dart_dict['corp_text'][idx])

In [5]:
# dart_txt = [dart['기업 이름'] + test(dart['contents']['II. 사업의 내용']['1. 사업의 개요'] 
#            test(dart['contents']['II. 사업의 내용']['2. 주요 제품 및 서비스'])
# #            test(dart['contents']['II. 사업의 내용']['6. 주요계약 및 연구개발활동'])
#             ) for dart in dart_data]

KeyError: 'II. 사업의 내용'

In [20]:
news_data['article']

0        중국 투자의향기업 직접 방문·투자협약 체결 협의이성훈 sinawi@hanmail.n...
1        이성훈 sinawi@hanmail.net포스코 광양제철소(소장 김학동)가 자동차 강...
2        이성훈 sinawi@hanmail.net여수광양항만공사(사장 방희석, 이하 공사)는...
3        지역 특화작목?틈새소득작목 발굴, 미래농업 준비이성훈 sinawi@hanmail.n...
4        2만2500여명 참여…생명나눔 확산이성훈 sinawi@hanmail.net포스코 광...
                               ...                        
27298    KCGI는 지분 늘려 13.47%로국민연금과 합치면 20% 넘어'백기사' 확보 등 ...
27299    [ 노유정 기자 ] 조양호 한진그룹 회장의 갑작스런 타계 소식이 전해진 8일 오전,...
27300    ] 조양호 한진그룹 회장이 타계하자 재계는 일제히 추모성명을 냈다.전국경제인연합회는...
27301    [ 김익환 기자 ] 조양호 한진그룹 회장이 타계하면서 그의 재산을 물려받을 장남 조...
27302    [ 김보형 기자 ]조양호 한진그룹 회장이 8일 타계하면서 그의 장남인 조원태 대한항...
Name: article, Length: 27303, dtype: object

In [26]:
news = news_data['article']

In [28]:
prop_news = [preprocessing(txt) for txt in news]

- 의미가 존재 [29,36,38]
- 언급되면 표시해줄 것 [42]
- 제주도나오면 반응 [45]
- 엔터[46]
- 농장[200]

In [52]:
target

'만여명 참여생명나눔 확산이성훈 포스코 광양제철소소장 김학동가 사랑나눔 헌혈행사를 실시하며 생명 나눔을 실천한지 올해로 년을 맞았다사랑나눔 헌혈행사는 광양제철소와 대한적십자사 광주전남혈액원이 함께 국가적인 혈액 부족 문제 해결에 적극 동참하고자 마련한 행사로 년 월부터 년 동안 광양제철소에서 정기적으로 이어오고 있다지난 일 열린 올해 첫사랑나눔 헌혈행사는 임직원들과 시민들의 왕래가 많은 제철소 내 생산종합관제센터와 금호동 복지센터에 마련된 헌혈 부스에서 진행됐으며 사전 사내 공지와 안내 메일 등을 통해 직원들의 헌혈 참여를 독려했다광양제철소는 헌혈 참여 직원들에게 봉사 마일리지와 함께 기념품 혹은 연말 소득공제 혜택 등을 제공하며 더 많은 임직원들이 헌혈을 통해 이웃 사랑을 실천할 수 있도록 돕고 있다이날 행사에는 광양제철소 직원 및 가족 등 총 여 명이 참여해 생명 나눔 활동에 적극 나서는 모습을 보였다헌혈에 참가한 한선교 사원은 품질기술부동절기를 맞아 혈액 보유량이 줄어들고 있다 들었다며 더 많은 직원들이 헌혈을 통해 사회 곳곳 생명이 위급한 환자들에게 도움이 되고 자신의 건강 상태도 검진해 보길 바란다 말했다임직원들의 적극적인 참여 결과 현재까지 광양지역 포스코패밀리 헌혈봉사 누적인원이 만여명을 기록했으며 전사적으로는 약 만여명에 달한다'

In [127]:
split_news=[]
for article in news_data['article']:
    split_news.append(list(map(preprocessing,article.split("."))))

In [130]:
target_idx=10
target = split_news[target_idx]
stopwords=['모두','지난','있다','회의','강조','성과','오는','모집','과정','계약',"체결","건립",'현대식','사업',
           '주관사로','통해','일까지','이벤트','지원','지역','기자','실험','한다','진행','설명회','예정이다',
           '이번','방문','올해','제공','찾아가','재림','당시','한국','프로젝트','있는','추진','다양한','적극',
           '위해','나갈','협력','협약','상당의','물품','명으로','지난해','증가했다','의한','영향','보면','통계',
           '순위','명당','관련','업체','부실','관심','명칭','합작법인','일자리','시흥배곧','서울대','했다',
           '한다고','위원장은','기반','혁신','인재','마련','정책','권고안','함께','시장','성장','대비','전월',
           '전년','동월','물가','하락','소비자','상승','운영','증가','억원','억만원','할인','사용','것으','위한',
           '방안','활성화','실무협의회','밝혔다','행위']
keywords = summarize_with_keywords(target, min_count=5, max_length=9,
    beta=0.5, max_iter=10,stopwords=stopwords,verbose=True)
tokenized_query = list(keywords.keys())
keywords

scan vocabs ... 
num vocabs = 57
done


{'대해': 1.557824048416248,
 '광양경제청': 1.3616088339018624,
 '국조실': 1.3169344526128928,
 '해외출장': 1.3042192065465863,
 '공사': 1.2014160144079296,
 '감리': 1.0,
 '경제': 0.618784478786961}

In [147]:
tokenized_query

['대해', '광양경제청', '국조실', '해외출장', '공사', '감리', '경제']

In [ ]:
doc_scores = bm25.get_scores(tokenized_query)

In [ ]:
list_dart_n = bm25.get_top_n(tokenized_zquery, list(pre_dart.values()), n=3)

In [ ]:
list_dart_n = bm25.get_top_n(tokenized_zquery, list(dart_dict.keys()), n=3)

In [20]:
from rank_bm25 import BM25Okapi
from krwordrank.word import summarize_with_keywords
target_idx=10
target = split_news[target_idx]
# tokenized_corpus = [doc.split(" ") for doc in dart_dict.keys()]
# bm25 = BM25Okapi(tokenized_corpus)
bm25 = BM25Okapi(list(pre_dart.values()))
# <rank_bm25.BM25Okapi at 0x1047881d0>

stopwords=['모두','지난','있다','회의','강조','성과','오는','모집','과정','계약',"체결","건립",'현대식','사업',
           '주관사로','통해','일까지','이벤트','지원','지역','기자','실험','한다','진행','설명회','예정이다',
           '이번','방문','올해','제공','찾아가','재림','당시','한국','프로젝트','있는','추진','다양한','적극',
           '위해','나갈','협력','협약','상당의','물품','명으로','지난해','증가했다','의한','영향','보면','통계',
           '순위','명당','관련','업체','부실','관심','명칭','합작법인','일자리','시흥배곧','서울대','했다',
           '한다고','위원장은','기반','혁신','인재','마련','정책','권고안','함께','시장','성장','대비','전월',
           '전년','동월','물가','하락','소비자','상승','운영','증가','억원','억만원','할인','사용','것으']
keywords = summarize_with_keywords(target, min_count=5, max_length=10,
    beta=0.85, max_iter=10,stopwords=stopwords,verbose=True)
tokenized_query = list(keywords.keys())

for key in ' '.join(target).split(" "):
    if key in dart_dict['corp_name']:
        print("언급된 기업 : ", key)

for key in tokenized_query:
    if key in dart_dict['corp_name']:
        print("언급된 기업 : ", key)
print(tokenized_query)
#tokenized_query = tokenizer.nouns(' '.join(target))
#tokenized_query = ' '.join(target).split(' ')
doc_scores = bm25.get_scores(tokenized_query)
# array([0.        , 0.93729472, 0.        ])

if max(doc_scores) <3:
    print("None")
else:
    result = []
    rank=1
    list_dart_n = bm25.get_top_n(tokenized_query, list(dart_dict.keys()), n=3)
    print("타겟뉴스")
    print(" ".join(target)[:1000])
    for dart in list_dart_n:
        print(f"{rank}순위 기업 : {dart_dict[dart]}")
        print(f"사업보고서 ::")
        print(dart[:500])
        rank+=1
    
# ['It is quite windy in London']

scan vocabs ... 
num vocabs = 76
done
['산업', '차위는', '농수산식품']
타겟뉴스
대통령직속 차산업혁명위원회는 일 서울 웨스틴조선호텔에서 차 산업혁명 글로벌 정책 컨퍼런스를 개최하고 차 산업혁명시대 정부 정책방향을 제시하는 차 산업혁명 대정부 권고안을 발표했다 차위는 사회산업지능화 기반 혁신 세 가지 축을 정책방향으로 제시했다 장병규 차위 위원장은 지난해 월 기 위원장직을 시작하면서 기 위원회의 핵심 업무로 민간 중심의 차 산업혁명 대정부 권고안을 마련하겠다고 했다며 지난 개월간 여명이 넘는 전문가들이 참여해 만들어진 차 산업혁명 대정부 권고안을 발표하게 됐다고 말했다 권고안은 지난 일 국무회의에 보고됐다 차 산업혁명 대정부 권고안 발표 기자간담회에서 장병규 차산업혁명위원회 위원장이 발표하고 있다 차위는 차 산업혁명 시대에 대응하기 위해 정부가 민간의 지원자 역할을 강화해야 한다고 했다 장 위원장은 정부가 혁신과 성장을 이끄는 인재를 양성하고 이들이 마음껏 도전할 수 있도록 노동 교육 사회보장 제도를 혁신해야 한다며 주 시간제의 일률적 적용 등 경직된 법 적용에서 탈피해 다양한 노동 형태를 포용할 수 있도록 노동 제도를 개선해야 한다고 제안했다 차 산업혁명 시대의 인재가 전통적 노동자와 다르다는 점도 강조했다 장 위원장은 인재는 생산수단을 스스로 소유하고 있다 네이버에 다니던 인재가 구글로 이직해도 거의 같은 개발 환경에서 근무할 수 있다라며 인재는 시간이 아닌 성과로 평가받으며 도전을 통해 차별화된 가치를 창출한다는 점에서 국제 경쟁력의 핵심요소라고 지적했다 차위는 혁신 인재를 성장할 수 있는 기반 마련을 위해 대학의 다양화와 재정 의사결정의 자율권 강화로 고등교육을 개혁할 기반을 만들어야 한다고 제안했다 차위는 산업 혁신 측면에서 경제효과가 큰 바이오헬스 제조 금융 스마트도시 모빌리티물류 등 지능화 혁신 개 분야와 미래비전 차원에서 농수산식품을 대 전략 분야로 선정했다 차위는 지능화 기반 혁신을 위해서는 기술데이터스타트업

In [187]:
from krwordrank.word import summarize_with_keywords
result = summarize_with_keywords(news[23], min_count=5, max_length=10,
    beta=0.85, max_iter=10, verbose=True)

scan vocabs ... 
num vocabs = 15
done


In [188]:
news

[['제주항공이 안전관리 체계 강화에 속도를 낸다',
  '제주항공은 현재 모의비행훈련장치 설치를 진행하고 있으며 장치의 성능과 운용품질 검사와 관계기관의 검사 등을 모두 마치고 이르면 다음달부터 자체 훈련 시작을 계획하고 있다고 일 밝혔다',
  '제주항공의 자체 모의비행훈련장치 도입은 국내 항공사에서 세 번째이며 비정상상황 회복훈련를 시현할 수 있는 것이 특징이다',
  '는 난기류 등의 요인으로 인해 항공기가 정상적인 자세를 유지하지 못할 경우를 대비한 훈련이다',
  '미국연방항공청 규정에 따라 미주노선을 운항하는 항공사의 조종사들은 반드시 이수해야 한다',
  '제주항공은 모의비행훈련장치를 도입해 안정적인 훈련 시스템을 구축함으로써 비행능력 향상은 물론 조종사를 채용할 때 지원자에 대한 높은 수준의 기량평가도 자체적으로 진행할 수 있게 된다',
  '안전에 대한 신뢰를 높이기 위한 외부 진단과 평가도 늘린다',
  '제주항공은 지난해 말 운항과 정비 객실과 운송 보안과 안전 등 항공 운송 전반에 걸친 국제항공운협회의 표준평가제도인      에 대한 재인증을 마친 데 이어 월 중 해외 컨설팅 업체인 프리즘을 통해 안전관리 절차와 품질에 대한 진단을 실시한다',
  '이에 앞서 제주항공은 지난해 월부터 운항품질관리를 위한 운항안전감사제도인     프로젝트를 진행하고 있다',
  '는 운항 특성을 분석해 잠재적 위험요인을 사전에 발견하고 이를 최적화한 표준 절차에 맞춰 위협 요인을 줄이는 프로그램이다',
  '전세계 개 이상의 항공사가  운영으로 운항안정성 확보에 나서고 있다',
  '증강현실과 가상현실 등 정보기술과 접목한 객실승무원 교육은 물론 업무 효율을 높이기 위한 통합 시스템 개선과 구축을 위한 투자도 확대한다',
  '제주항공 관계자는 적절한 시설 투자와 외부 진단 시스템 개선 등을 통해 이용자들이 신뢰할 수 있는 안전운항 체계를 만들어 가겠다고 말했다',
  '사진제주항공'],
 ['대구 상수도관 절반 이상 노후화로 잦은 파열 일으켜',
  '

In [176]:
result.keys()

dict_keys(['연구비', '관리', '평가'])

In [15]:
out_list = []

In [18]:
with open("out_list.pickle","wb") as f:
    pickle.dump(out_list,f)

In [17]:
import pickle

In [19]:
with open("out_list.pickle","rb") as f:
    d = pickle.load(f)

In [20]:
d

[]

In [22]:
data1

,summarized,original,article,abstract
0,여수광양항만공사(사장 방희석)는 14일부터 20일까지 광양항 배후단지 투자 유치를 ...,"['중국 투자의향기업 직접 방문·투자협약 체결 협의', '이성훈 sinawi@han...",중국 투자의향기업 직접 방문·투자협약 체결 협의이성훈 sinawi@hanmail.n...,['여수광양항만공사는 광양만권경제자유구역청과 합동으로 직접 중국 현지의 투자의향기업...
1,광양제철소는 지난 16일 3냉연공장 합리화 준공식을 열었다.광양제철소가 최신설비로 ...,"['이성훈 sinawi@hanmail.net', '포스코 광양제철소(소장 김학동)가...",이성훈 sinawi@hanmail.net포스코 광양제철소(소장 김학동)가 자동차 강...,['포스코 광양제철소가 자동차 강판 생산공장인 제3냉연공장을 최신 설비로 재탄생시키...
2,"여수광양항만공사(사장 방희석, 이하 공사)는 2018년도 승진 및 전보 인사를 단행...","['이성훈 sinawi@hanmail.net', '여수광양항만공사(사장 방희석, 이...","이성훈 sinawi@hanmail.net여수광양항만공사(사장 방희석, 이하 공사)는...",['여수광양항만공사는 광양항 활성화 및 물동량 창출을 위해 인력을 보강하고 ‘가치경...
3,광양시는 경쟁력을 갖춘 풍요로운 농어촌 건설을 위해 원예작물 신기술보급사업을 통한 ...,"['지역 특화작목?', '틈새소득작목 발굴, 미래농업 준비', '이성훈 sinawi...","지역 특화작목?틈새소득작목 발굴, 미래농업 준비이성훈 sinawi@hanmail.n...",['광양시는 경쟁력을 갖춘 농어촌 건설을 위해 지역 특화작목을 비롯한 미래농업 분야...
4,포스코 광양제철소(소장 김학동)가 ‘사랑나눔 헌혈행사’를 실시하며 생명 나눔을 실천...,"['2만2500여명 참여…생명나눔 확산', '이성훈 sinawi@hanmail.ne...",2만2500여명 참여…생명나눔 확산이성훈 sinawi@hanmail.net포스코 광...,"[""20년 째 '사랑나눔 헌혈행사'를 진행해온 포스코 광양제철소는 직원들이 헌혈을 ..."
...,...,...,...,...
27298,한진칼 2대주주인 강성부 펀드(KCGI)의 경영권 위협은 더욱 거세질 전망이다.금융...,"['KCGI는 지분 늘려 13.47%로', '국민연금과 합치면 20% 넘어', ""'...",KCGI는 지분 늘려 13.47%로국민연금과 합치면 20% 넘어'백기사' 확보 등 ...,['8일 IB업계 관계자는 상속세 납부 과정에서 조 회장 일가의 한진칼 지배구조 약...
27299,"[ 노유정 기자 ] 조양호 한진그룹 회장의 갑작스런 타계 소식이 전해진 8일 오전,...",['[ 노유정 기자 ] 조양호 한진그룹 회장의 갑작스런 타계 소식이 전해진 8일 오...,"[ 노유정 기자 ] 조양호 한진그룹 회장의 갑작스런 타계 소식이 전해진 8일 오전,...",['8일 미국에서 들려온 조양호 한진그룹 회장의 사망 소식에 서울 공항동 대한항공 ...
27300,] 조양호 한진그룹 회장이 타계하자 재계는 일제히 추모성명을 냈다.전국경제인연합회는...,"['] 조양호 한진그룹 회장이 타계하자 재계는 일제히 추모성명을 냈다.', '전국경...",] 조양호 한진그룹 회장이 타계하자 재계는 일제히 추모성명을 냈다.전국경제인연합회는...,"[""조양호 한진그룹 회장이 타계하고 8일 전국경제인연합회는 논평을 통해 한국 항공과..."
27301,8일 한국거래소에 따르면 조 회장이 보유한 그룹 지주회사 한진칼(지분율 17.84%...,['[ 김익환 기자 ] 조양호 한진그룹 회장이 타계하면서 그의 재산을 물려받을 장남...,[ 김익환 기자 ] 조양호 한진그룹 회장이 타계하면서 그의 재산을 물려받을 장남 조...,"['고 조 회장의 한진칼, 한진 등 상장회사의 주식가격 총합은 약 3560억 정도 ..."
